In [1]:
import numpy as np
import torch 
import os
import sys
import pandas as pd
import numpy as np
import glob
import re
from pprint import pprint
from typing import *
sys.path.append('./..')

from PanjivaDataPreprocessor import data_fetcher_v1

In [2]:

record_data = data_fetcher_v1.get_training_set_data('us_import1')
del record_data['PanjivaRecordID']
data = record_data.head(200)
record_data = data.values + 1
domain_dims_df = data_fetcher_v1.get_domain_dims('us_import1')
cardinality = domain_dims_df['dimension'].tolist()
cardinality = [ _ + 1 for _ in cardinality]


In [3]:
record_data = record_data[:15]
record_data

array([[ 207, 1055, 1114,  233,   37,   83,   16, 1243],
       [  80,  238,  530,  185,   37,   84,   16, 1778],
       [ 530, 2370, 1211,  233,    8,   21,   16,  934],
       [ 279, 2368, 1471,  185,    8,   21,   16,  910],
       [ 310,  723, 1471,  260,    8,   21,   16,  597],
       [ 279, 2368, 1471,   98,   16,   41,   16, 1298],
       [  81, 1471, 1318,  233,   39,   85,   16,  238],
       [  26,  723, 1374,  233,   39,   85,   16,  545],
       [ 345,  622,  552,  233,   46,   93,   16,  532],
       [  95,  571,  802,  233,   58,  120,   16,  219],
       [ 199,   53,  969,  233,   39,  102,   16,  156],
       [ 471, 1072, 1325,  260,   59,  121,   16,  651],
       [ 207,  415, 1181,  260,   59,  121,   16, 2016],
       [ 355, 1368,  603,  233,   13,   37,   16, 1496],
       [  76, 2121, 1471,  233,   24,   60,   16, 1728]])

In [4]:
'''
Set the MASK token to be 0
token weights = 0 - means the loss does not need to be accounted for 
'''



data = record_data
# def get_maksed_tokens(
#     data, 
#     column_cardinalities, 
#     k1 = 0.8, 
#     k2 = 0.2,
#     uniform_column_choice = True
# ):
    
    

column_cardinalities = cardinality
k1 = 0.8
k2 = 0.2
uniform_column_choice = True
    
    
column_indices = np.array(data)
mask_token_id = 0

# =======
num_attr = data.shape[-1]
if uniform_column_choice:
    column_prob = np.array([1/num_attr for _ in range(num_attr)])
else:
    column_prob = np.array([np.power(column_cardinalities[i]/sum(column_cardinalities),0.5) for i in range(num_attr)]) 
    column_prob = column_prob/np.sum(column_prob)
print('[Columns Probability]',column_prob)    

inp_mask = np.random.multinomial(1, column_prob, column_indices.shape[:1])
inp_mask = np.array(inp_mask, dtype=bool)
print('column_indices.shape', column_indices.shape)
# ======
# Set targets to -1 by default, it means ignore
# the labels to be predicted 
# ======

[Columns Probability] [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
column_indices.shape (15, 8)


In [5]:
inp_mask

array([[False, False, False, False,  True, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False,  True, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False,  True, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [ True, False, False, False, False, False, False, False],
       [False, False,  True, False, False, False, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False,  True]])

In [6]:
column_indices

array([[ 207, 1055, 1114,  233,   37,   83,   16, 1243],
       [  80,  238,  530,  185,   37,   84,   16, 1778],
       [ 530, 2370, 1211,  233,    8,   21,   16,  934],
       [ 279, 2368, 1471,  185,    8,   21,   16,  910],
       [ 310,  723, 1471,  260,    8,   21,   16,  597],
       [ 279, 2368, 1471,   98,   16,   41,   16, 1298],
       [  81, 1471, 1318,  233,   39,   85,   16,  238],
       [  26,  723, 1374,  233,   39,   85,   16,  545],
       [ 345,  622,  552,  233,   46,   93,   16,  532],
       [  95,  571,  802,  233,   58,  120,   16,  219],
       [ 199,   53,  969,  233,   39,  102,   16,  156],
       [ 471, 1072, 1325,  260,   59,  121,   16,  651],
       [ 207,  415, 1181,  260,   59,  121,   16, 2016],
       [ 355, 1368,  603,  233,   13,   37,   16, 1496],
       [  76, 2121, 1471,  233,   24,   60,   16, 1728]])

In [7]:
labels = -1 * np.ones(column_indices.shape, dtype=int)
labels[inp_mask] = column_indices[inp_mask]

column_indices_masked = np.copy(column_indices)
# Replace k1 fraction of tokens with [MASK] following MLM

inp_mask_2mask = inp_mask & (np.random.rand(*column_indices_masked.shape) < k1)
column_indices_masked[inp_mask_2mask] = mask_token_id  

# The indices that are not going to be changed
# Set this fraction to k2

inp_mask_2random = inp_mask_2mask & (np.random.rand(*column_indices_masked.shape) < k2)

In [8]:
inp_mask_2random[:]

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False,  True, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [9]:
column_indices_masked

array([[ 207, 1055, 1114,  233,   37,   83,   16, 1243],
       [  80,  238,  530,  185,    0,   84,   16, 1778],
       [ 530, 2370, 1211,  233,    8,   21,   16,  934],
       [ 279,    0, 1471,  185,    8,   21,   16,  910],
       [ 310,  723, 1471,  260,    0,   21,   16,  597],
       [ 279,    0, 1471,   98,   16,   41,   16, 1298],
       [  81,    0, 1318,  233,   39,   85,   16,  238],
       [  26,  723,    0,  233,   39,   85,   16,  545],
       [ 345,  622,  552,  233,    0,   93,   16,  532],
       [   0,  571,  802,  233,   58,  120,   16,  219],
       [ 199,   53,    0,  233,   39,  102,   16,  156],
       [ 471, 1072, 1325,  260,   59,  121,    0,  651],
       [ 207,  415, 1181,  260,   59,  121,   16, 2016],
       [ 355, 1368,  603,  233,    0,   37,   16, 1496],
       [  76, 2121, 1471,  233,   24,   60,   16, 1728]])

In [10]:
# ----------------------
# Replace masks in each column by randomly sampling from candidate values
# ----------------------
for i in range(num_attr):
    _values= np.random.randint(1, column_cardinalities[i]+1, inp_mask_2random[:,i].sum())
    column_indices_masked[:,i][inp_mask_2random[:,i]] = _values
    
data_masked = column_indices_masked
token_weights = np.ones(labels.shape)
token_weights[labels == -1] = 0

0 Values [] 532
[207  80 530 279 310 279  81  26 345   0 199 471 207 355  76]
[207  80 530 279 310 279  81  26 345   0 199 471 207 355  76]
1 Values [990] 2386
[1055  238 2370    0  723    0    0  723  622  571   53 1072  415 1368
 2121]
[1055  238 2370    0  723  990    0  723  622  571   53 1072  415 1368
 2121]
0 Values [] 1547
[1114  530 1211 1471 1471 1471 1318    0  552  802    0 1325 1181  603
 1471]
[1114  530 1211 1471 1471 1471 1318    0  552  802    0 1325 1181  603
 1471]
0 Values [] 264
[233 185 233 185 260  98 233 233 233 233 233 260 260 233 233]
[233 185 233 185 260  98 233 233 233 233 233 260 260 233 233]
3 Values [43 51 13] 61
[37  0  8  8  0 16 39 39  0 58 39 59 59  0 24]
[37 43  8  8  0 16 39 39 51 58 39 59 59 13 24]
0 Values [] 127
[ 83  84  21  21  21  41  85  85  93 120 102 121 121  37  60]
[ 83  84  21  21  21  41  85  85  93 120 102 121 121  37  60]
1 Values [58] 85
[16 16 16 16 16 16 16 16 16 16 16  0 16 16 16]
[16 16 16 16 16 16 16 16 16 16 16 58 16 16 16]
0 V

In [12]:
token_weights

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [13]:
labels

array([[  -1,   -1,   -1,   -1,   37,   -1,   -1,   -1],
       [  -1,   -1,   -1,   -1,   37,   -1,   -1,   -1],
       [  -1,   -1,   -1,  233,   -1,   -1,   -1,   -1],
       [  -1, 2368,   -1,   -1,   -1,   -1,   -1,   -1],
       [  -1,   -1,   -1,   -1,    8,   -1,   -1,   -1],
       [  -1, 2368,   -1,   -1,   -1,   -1,   -1,   -1],
       [  -1, 1471,   -1,   -1,   -1,   -1,   -1,   -1],
       [  -1,   -1, 1374,   -1,   -1,   -1,   -1,   -1],
       [  -1,   -1,   -1,   -1,   46,   -1,   -1,   -1],
       [  95,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
       [  -1,   -1,  969,   -1,   -1,   -1,   -1,   -1],
       [  -1,   -1,   -1,   -1,   -1,   -1,   16,   -1],
       [  -1,   -1,   -1,   -1,   59,   -1,   -1,   -1],
       [  -1,   -1,   -1,   -1,   13,   -1,   -1,   -1],
       [  -1,   -1,   -1,   -1,   -1,   -1,   -1, 1728]])

In [3]:
return data_masked, labels, token_weights

IndentationError: unexpected indent (3401611849.py, line 26)

In [ ]:
'''
Sample run
'''
# column_indices = [
#     [12,90,64,24, 240],
#     [22,91,65,25,241],
#     [32,92,66,26,242],
#     [42,93,67,27,243]
# ]
# column_indices = np.array(column_indices)
# column_cardinalities = [45,100,70,30,250]

# get_maksed_tokens(
#     column_indices, 
#     column_cardinalities,
#     uniform_column_choice = False
# )